In [1]:
def analytical(matrix, rhs, objcoef, constant=0, direction="max"):
    '''Analytical method to solve a linear programming problem

    This function solves linear programming problems through trial and error method (analytical method). This method uses a greedy approch by solving all linear equation subsystems. The total number of basic solutions are the combinations of m equations in m+n variables, where m + n variables are the total number of variables whenever the standard form for linear programs are used.

    Parameters
    ----------
    matrix: numpy.ndarray
       Body matrix of the linear programming problem (left-hand side)

    rhs: numpy.ndarray
        The right-hand side of the linear programming problem

    objcoef: numpy.ndarray
        Objective function coefficients in the standard form. The zeros corresponding to slack or surplus variables must be taken into account.

    constant: scalar, default 0
        If the objective function has an independent term, this one must be included as part of the objective function.

    direction: {"max", "min"}
        Sense of the objective function (the default is "max", which implies a maximization problem)

    Returns
    ----------
    feasible_solutions: numpy ndarray
        Matrix with feasible solutions

    infeasible_solutions: numpy ndarray
        Matrix with infeasible solutions

    best_vector: numpy ndarray
        Array with the best solutions. This array includes z value.


    '''
    num_rows, num_cols = matrix.shape
    lst_feasible = []    # list of feasible solutions
    lst_infeasible = []
    for variables in combinations(range(num_cols), num_rows):
        try:
            vector = np.zeros(num_cols)   # zero vector  [0, 0, 0, ... , 0]
            basic = np.linalg.solve(matrix[:, variables], rhs)
            vector[list(variables)] = basic
            z = objcoef.dot(vector) + constant
            solution = np.concatenate([vector, [z]])   # [x1, x2, x,3] + [zvalue] = [x1, x2, x3, zvalue]
            if np.any(basic < 0):   # infeasible
                lst_infeasible.append(solution)
            else:
                lst_feasible.append(solution)  # feasible solutions
        except np.linalg.LinAlgError:
            print(f"Singular Matrix with variables {variables}")
    feasible_solutions = np.array(lst_feasible)
    infeasible_solutions = np.array(lst_infeasible)  # array of infeasble solutions
    if direction == "max":
        best_index = np.argmax(feasible_solutions[:, -1])  # last column with z value
    else:
        best_index = np.argmin(feasible_solutions[:, -1])
    best_vector = feasible_solutions[best_index]
    return feasible_solutions, infeasible_solutions,  best_vector

In [3]:
def simplex(matrix, rhs, z, inequalities, direction=1, M=1000, vlabel="x"):
    '''Simplex algorithm to solve linear programming problems

    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side

    rhs: numpy ndarray
        Right-hand side vector

    inequalities: list of strings
        List with inequality strings

    M: int, default=1000
       Penalty quantity

    direction: {+1 , -1}
        For maximization problems use +1 and for minimization problems use -1 instead.
    '''
    print("=" * 20, "Simplex Method", "=" * 20)
    if not isinstance(matrix, np.ndarray):
        matrix = np.array(matrix, dtype=float)

    numconstraints, numxvars = matrix.shape
    rhs = np.array(rhs, dtype=float)

    matrix, labels, z = create_fullmatrix(matrix, inequalities, z, vlabel, direction, M)

    num_rows, num_cols = matrix.shape
    
    cb_index = np.where((matrix == 1) & (np.abs(matrix).sum(axis=0) == 1))[1]
    
    
    cb = z[cb_index]
    
    zj = cb.dot(matrix)

    net_evaluation = direction * (z - zj)

    solutions = []
    fvalues = []

    iteration = 0
    while np.any(net_evaluation > 0):
        solution = np.zeros_like(z)
        entering = net_evaluation.argmax()  # entering variables (index)

        key_col = matrix[ : , entering]
        ratios = np.divide(rhs, key_col, out=np.full_like(rhs, np.inf), where=key_col>0)
        leaving = ratios.argmin()   # leaving variables (index)

        pivot = matrix[leaving, entering]

        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot

        for i in range(num_rows):
            if i == leaving:
                continue
            factor = matrix[i, entering]
            matrix[i] = -factor * matrix[leaving] + matrix[i]
            rhs[i] = -factor * rhs[leaving] + rhs[i]

        leaving_label = labels[cb_index[leaving]]
        entering_label= labels[entering]

        cb_index[leaving] = entering

        cb = z[cb_index]
        zj = cb.dot(matrix)
        basic_labels = labels[cb_index]
        net_evaluation = direction * (z - zj)

        solution[cb_index] = rhs  # basics

        iteration += 1

        
        print("#" * 20)
        print(f"Iteration {iteration}. {leaving_label} --> {entering_label}")
        print("-" * 20)
        print(matrix,  "\n")
        print(pd.DataFrame(cb, index=basic_labels, columns=["Basis"]))
        print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}", "\n")

        solutions.append(solution)
        fvalues.append(cb.dot(rhs))
        if np.all(net_evaluation <= 0):
            print("#" * 20)
            print(f"Optimal solution found in {iteration} iterations")
            print(pd.DataFrame(solution, index=labels, columns=["Solution"]))
            print("\nOptimal Table:")
            print(
                pd.DataFrame(np.hstack((cb[:,np.newaxis], matrix, rhs[:,np.newaxis])), index=basic_labels, columns=["Cb"] + labels.tolist() + ["b"])
                 )
            print("\nRow Base:")
            print(
                pd.DataFrame(
                np.vstack((zj, direction * net_evaluation)), 
                    index=["zj", "cj - zj"],
                    columns=labels,
                )
                 )
    return np.array(solutions), fvalues, np.vstack((zj, direction*net_evaluation)), matrix